In [ ]:
import cv2
import json
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
data_dir = Path('../data/test/')
subset = 'a'
json_ref_file = data_dir / 'b_annotations.json'
json_pred_file = Path('../submit/out_top_5.json')

In [ ]:
with open(str(json_pred_file), 'r') as f:
    preds = json.load(f)

imgs = preds['images']
annotations = preds['annotations']

with open(str(json_ref_file), 'r') as f:
    refs = json.load(f)
    
categories = refs['categories']

In [ ]:
num_classes = len(categories)
classes = []
class_ids = []
for category in categories:
    classes.append(category['name'])
    class_ids.append(category['id'])

class_ids = np.array(class_ids)
classes = np.array(classes)

In [ ]:
def get_image_annotation(index):
    img_info = imgs[index]
    img_name = img_info['file_name']
    img_path = data_dir / '{}_images'.format(subset) / img_name
    img_id = img_info['id']
    bbox_list = []
    cls_list = []
    for gt in annotations:
        if gt['image_id'] == img_id:
            bbox = gt['bbox']
            cls_id = gt['category_id']
            bbox_list.append(bbox)
            cls_list.append(cls_id)
    return {'img_path': img_path, 'bbox_list': bbox_list, 'cls_list': cls_list}

In [ ]:
colors = np.random.randint(0, 255, size=(classes.shape[0], 3), dtype=np.uint8)
plt.rcParams['figure.figsize'] = [22, 15]

def show_example(example_index):
    gt_dict = get_image_annotation(example_index)
    img_path = gt_dict['img_path']
    bbox_list = gt_dict['bbox_list']
    cls_list = gt_dict['cls_list']
    img = cv2.imread(str(img_path))
    for i, bbox in enumerate(bbox_list):
        cls_id = cls_list[i]
        cls_name = classes[cls_id]
        x1, y1, w, h = bbox
        color = colors[cls_id].tolist()
        cv2.rectangle(img, (x1, y1), (x1+w, y1+h), color, 3)
        cv2.putText(img, cls_name, (x1+10, y1 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    print(img.shape)
    plt.show()

In [ ]:
show_example(103)